In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *


In [2]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[2]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [6]:
import time

# TODO: fix this hard-coded path
embeddingsFile = '/home/jose/Downloads/bio_nlp_vec/PubMed-shuffle-win-2.bin'

documentAssembler = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")\

assertion = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols(["document"])\
    .setOutputCol("assertion")\
    .setBefore(11)\
    .setAfter(13)\
    .setEmbeddingsSource(embeddingsFile,200,3)


finisher = Finisher() \
    .setInputCols(["assertion"]) \
    .setIncludeKeys(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    assertion,
    finisher
  ])


In [7]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/negex.parquet"). \
        limit(3000)
data.cache()
data.count()
data.show()

+--------------------+--------------------+--------+-----+---+
|            sentence|              target|   label|start|end|
+--------------------+--------------------+--------+-----+---+
|**initials ______...|multinodular goit...|Affirmed|   21| 25|
|02) mild aortic r...|mild aortic regur...|Affirmed|    1|  3|
|02) mild left atr...|mild left atrial ...|Affirmed|    1|  4|
|02) mild left atr...|mild left atrial ...|Affirmed|    1|  4|
|02) mild to moder...|mild to moderate ...|Affirmed|    1|  5|
|02) mild to moder...|mild to moderate ...|Affirmed|    1|  5|
|02) no valvular a...|valvular abnormal...| Negated|    2|  3|
|02) nondilated ri...|nondilated right ...|Affirmed|    1|  9|
|02) normal left v...|normal left ventr...|Affirmed|    1|  4|
|02) normal left v...|normal left ventr...|Affirmed|    1|  6|
|02) paradoxical s...|post-operative se...|Affirmed|    6|  8|
|02) small left ve...|small left ventri...|Affirmed|    1|  8|
|03) mild mitral r...|mild mitral regur...|Affirmed|   

In [8]:
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")

Start fitting
Fitting is ended


In [11]:
result = model.transform(data)
result.select("sentence", "target", "finished_assertion").show()

+--------------------+--------------------+------------------+
|            sentence|              target|finished_assertion|
+--------------------+--------------------+------------------+
|**initials ______...|multinodular goit...|  result->Affirmed|
|02) mild aortic r...|mild aortic regur...|  result->Affirmed|
|02) mild left atr...|mild left atrial ...|  result->Affirmed|
|02) mild left atr...|mild left atrial ...|  result->Affirmed|
|02) mild to moder...|mild to moderate ...|  result->Affirmed|
|02) mild to moder...|mild to moderate ...|  result->Affirmed|
|02) no valvular a...|valvular abnormal...|   result->Negated|
|02) nondilated ri...|nondilated right ...|  result->Affirmed|
|02) normal left v...|normal left ventr...|  result->Affirmed|
|02) normal left v...|normal left ventr...|  result->Affirmed|
|02) paradoxical s...|post-operative se...|  result->Affirmed|
|02) small left ve...|small left ventri...|  result->Affirmed|
|03) mild mitral r...|mild mitral regur...|  result->Af

In [10]:
pipeline.write().overwrite().save("./assertion_pipeline")
model.write().overwrite().save("./assertion_model")

Py4JJavaError: An error occurred while calling o286.save.
: scala.NotImplementedError: The default jsonEncode only supports string and vector. org.apache.spark.ml.param.Param must override jsonEncode for java.lang.Integer.
	at org.apache.spark.ml.param.Param.jsonEncode(params.scala:98)
	at org.apache.spark.ml.util.DefaultParamsWriter$$anonfun$1$$anonfun$2.apply(ReadWrite.scala:296)
	at org.apache.spark.ml.util.DefaultParamsWriter$$anonfun$1$$anonfun$2.apply(ReadWrite.scala:295)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.ml.util.DefaultParamsWriter$$a

In [9]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./assertion_pipeline")
sameModel = PipelineModel.read().load("./assertion_model")